In [40]:
import warnings
from IPython.core.interactiveshell import InteractiveShell
from bs4 import BeautifulSoup

# https://github.com/WillKoehrsen/recurrent-neural-networks/blob/master/notebooks/Deep%20Dive%20into%20Recurrent%20Neural%20Networks.ipynb
InteractiveShell.ast_node_interactivity = 'all'

warnings.filterwarnings('ignore', category=RuntimeWarning)

RANDOM_STATE = 50
EPOCHS = 100
BATCH_SIZE = 2048
TRAINING_LENGTH = 20
TRAIN_FRACTION = 0.7
LSTM_CELLS = 16
VERBOSE = 1
SAVE_MODEL = True

In [41]:
import pandas as pd
import numpy as np
import pickle

In [42]:
anime_data = pd.read_csv("./data/anime_dataset.csv")

In [43]:
anime_data.head(1)

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,Link
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop


In [44]:
anime_synopsis_column_name = "Synopsis"
anime_genre_column_name = "Genre"

In [45]:
anime_data.drop(anime_data[anime_data[anime_synopsis_column_name].isnull()].index, inplace=True)
anime_data.drop(anime_data[anime_data[anime_genre_column_name].isnull()].index, inplace=True)
anime_data[anime_synopsis_column_name].isnull().sum()
anime_data[anime_genre_column_name].isnull().sum()

0

0

In [46]:
anime_synopsis = list(anime_data[anime_synopsis_column_name])

# Data cleaning

In [47]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


This function “decontracted” defined above takes a text column from a data frame and removes all HTML tags and special characters.

In [48]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
i = 0
for sentance in anime_data[anime_synopsis_column_name].values:
    # some values might be simply links
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    i += 1


## Prepare for training

In [53]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)

def remove_out_brackets(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.replace("[","")
        item=item.replace("]","")
        nospace.append(item)
    return (",").join(nospace)

def remove_quote_mark(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.replace("'","")
        nospace.append(item)
    return (",").join(nospace)
    


In [54]:
anime_data[anime_genre_column_name].isnull().sum()

0

In [57]:
anime_data[anime_genre_column_name]=anime_data[anime_genre_column_name].apply(remove_spaces)
anime_data[anime_genre_column_name]=anime_data[anime_genre_column_name].apply(remove_out_brackets)
anime_data[anime_genre_column_name]=anime_data[anime_genre_column_name].apply(remove_quote_mark)

In [58]:
anime_data['Genre'].head(5)

0      Adventure,Comedy,Drama,Sci-Fi
1                Space,Drama,Mystery
2      Sci-Fi,Adventure,Comedy,Drama
3    Magic,Police,Supernatural,Drama
4                    Fantasy,Shounen
Name: Genre, dtype: object

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

In [64]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(anime_data[anime_genre_column_name]).toarray()
y_train.shape

print(f"We have a total of {y_train.shape[1]} genres for {y_train.shape[0]} animes.")

(13743, 41)

We have a total of 41 genres for 13743 animes.


In [65]:
y_train

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])